In [ ]:
from LayerByLayerOptimization import *
import visualization
import matplotlib

## Set up the tensor product simulator

In [ ]:
DIM=3

BC_PATH = '../examples/bcs/3D/hump.bc'
domainCorners = [[0, 0, 0], [2, 1, 1]]
gridResolution = [64, 32, 32]

tps = pyVoxelFEM.TensorProductSimulator([1] * DIM, domainCorners, gridResolution)
tps.readMaterial('../examples/materials/B9Creator.material')
tps.applyDisplacementsAndLoadsFromFile(BC_PATH)

## Set up the main objective

In [ ]:
numCoaseningLevels = optMGLevel(tps.NbElementsPerDimension, DIM) # Number of optimal coasening levels
optObj = pyVoxelFEM.MultigridComplianceObjective(tps.multigridSolver(numCoaseningLevels))
optObj.mgSmoothingIterations = 1
optObj.tol = 1e-5

## Set up the Layer-by-Layer objective

In [ ]:
layObj = LayerByLayerObjective(tps, mg_levels=numCoaseningLevels, init_method='N=3')
layObj.setCGTol(1e-5)
layObj.zeroInit = False # if false, the full object simulation will use the result of its previous optimization

## Define the Layer-by-Layer optimization problem

In [ ]:
weight = 10
maxVol = 0.1
constraints = [pyVoxelFEM.TotalVolumeConstraint(maxVol)] # Volume constraint

# Set up the chain of density filters
filters = [pyVoxelFEM.SmoothingFilter(radius=2, type=pyVoxelFEM.SmoothingFilter.Type.Linear), pyVoxelFEM.ProjectionFilter(beta=5)] # Smoothing filter followed by projection

LayerByLayerOptimizationProblem = getClass(eval(pyVoxelFEM.getClassName(tps, "TopologyOptimizationProblem")))

top = LayerByLayerOptimizationProblem(tps, optObj, constraints, filters, layObj, float(weight))
top.setVars(filters[-1].invert(maxVol) * np.ones(top.numVars()));

# BC visualization

In [ ]:
view = visualization.TPSViewer(tps)
view.show()

In [ ]:
# Hide the voxels below a given threshold
view.densityThreshold = 0.5 # None t disable threshold

In [ ]:
max_iters=50
optimizer = MMA(top, max_iters, callback = lambda *_: view.update())
benchmark.reset()
optimizer.run(top.getDensities(), showProgress=True)
benchmark.report()

In [ ]:
view.show()

In [ ]:
# Change colormap
view.densityColormap = matplotlib.cm.jet